In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader

import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
batch_size = 100
learning_rate = 0.0002
num_epoch = 10

In [ ]:
mnist_train = datasets.MNIST(roo="../Data/",train=True,transform = transforms.ToTensor(),target_tramsform=None, download=True)
mnist_test = datasets.MNIST(roo="../Data/",train=False,transform = transforms.ToTensor(),target_tramsform=None, download=True)

In [ ]:
train_loader = DataLoader(mnist_train,batch_size=batch_size,shuffle=True,num_workers=2,drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size,shuffle=False,num_workers=2,drop_last=True)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.layer = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5),
            nn.ReLU(),

            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5),
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=2,stride=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5),
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=2,stride=2)
        )   
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3,100),
            nn.ReLU(),
            nn.Linear(100,10)
        )

    def forward(self,x):
        out = self.layer(x)
        out = out.view(batch_size,-1)